# Encrypted Inference Example

In [1]:
%install-location $cwd/swift-install
%install '.package(url: "https://github.com/tensorflow/swift-models", .branch("master"))' Datasets
%install '.package(path: "$cwd/ppmlNB_tensor")' ppmlNB_tensor

Installing packages:
	.package(url: "https://github.com/tensorflow/swift-models", .branch("master"))
		Datasets
	.package(path: "/root/swift-ppml/secure_computation/ppmlNB_tensor")
		ppmlNB_tensor
With SwiftPM flags: []
Working in: /tmp/tmpkxufudx6/swift-install
[1/2] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
[2/3] Merging module jupyterInstalledPackages
Initializing Swift...
Installation complete!


In [2]:
import ppmlNB_tensor
import TensorFlow

## Public Training

In [3]:
import Datasets

let epochCount = 12
let batchSize = 128

let dataset = MNIST()

var classifier = Sequential {
    Flatten<Float>()
    Dense<Float>(inputSize: 784, outputSize: 10)
}

let optimizer = SGD(for: classifier, learningRate: 0.1)

print("Beginning training...")

struct Statistics {
    var correctGuessCount: Int = 0
    var totalGuessCount: Int = 0
    var totalLoss: Float = 0
    var batches: Int = 0
}

let testBatches = dataset.testDataset.batched(batchSize)

// The training loop.
for epoch in 1...epochCount {
    var trainStats = Statistics()
    var testStats = Statistics()
    let trainingShuffled = dataset.trainingDataset.shuffled(
        sampleCount: dataset.trainingExampleCount, randomSeed: Int64(epoch))

    Context.local.learningPhase = .training
    for batch in trainingShuffled.batched(batchSize) {
        let (labels, images) = (batch.label, batch.data)
        // Compute the gradient with respect to the model.
        let 𝛁model = TensorFlow.gradient(at: classifier) { classifier -> Tensor<Float> in
            let ŷ = classifier(images)
            let correctPredictions = ŷ.argmax(squeezingAxis: 1) .== labels
            trainStats.correctGuessCount += Int(
                Tensor<Int32>(correctPredictions).sum().scalarized())
            trainStats.totalGuessCount += batchSize
            let loss = softmaxCrossEntropy(logits: ŷ, labels: labels)
            trainStats.totalLoss += loss.scalarized()
            trainStats.batches += 1
            return loss
        }
        // Update the model's differentiable variables along the gradient vector.
        optimizer.update(&classifier, along: 𝛁model)
    }

    Context.local.learningPhase = .inference
    for batch in testBatches {
        let (labels, images) = (batch.label, batch.data)
        // Compute loss on test set
        let ŷ = classifier(images)
        let correctPredictions = ŷ.argmax(squeezingAxis: 1) .== labels
        testStats.correctGuessCount += Int(Tensor<Int32>(correctPredictions).sum().scalarized())
        testStats.totalGuessCount += batchSize
        let loss = softmaxCrossEntropy(logits: ŷ, labels: labels)
        testStats.totalLoss += loss.scalarized()
        testStats.batches += 1
    }

    let trainAccuracy = Float(trainStats.correctGuessCount) / Float(trainStats.totalGuessCount)
    let testAccuracy = Float(testStats.correctGuessCount) / Float(testStats.totalGuessCount)
    print(
        """
        [Epoch \(epoch)] \
        Training Loss: \(trainStats.totalLoss / Float(trainStats.batches)), \
        Training Accuracy: \(trainStats.correctGuessCount)/\(trainStats.totalGuessCount) \
        (\(trainAccuracy)), \
        Test Loss: \(testStats.totalLoss / Float(testStats.batches)), \
        Test Accuracy: \(testStats.correctGuessCount)/\(testStats.totalGuessCount) \
        (\(testAccuracy))
        """)
}

Loading resource: train-images-idx3-ubyte
Loading resource: train-labels-idx1-ubyte
2020-02-05 03:47:51.219991: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 188160000 exceeds 10% of system memory.
2020-02-05 03:47:51.283980: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2020-02-05 03:47:51.311763: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2208000000 Hz
2020-02-05 03:47:51.314070: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1fd44f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-02-05 03:47:51.314169: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-02-05 03:47:51.318478: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 188160000 exceeds 10% of system memory.
Loading resource: t1

In [4]:
let w1 = classifier.layer2.weight
let b1 = classifier.layer2.bias

In [5]:
func get_input() -> (Tensor<Float>, Tensor<Int32>){
    let dataset = MNIST()
    let trainingShuffled = dataset.trainingDataset.shuffled(
        sampleCount: dataset.trainingExampleCount, randomSeed: Int64(1))
    var train_iter = trainingShuffled.self.makeIterator()
    var image = train_iter.next().unsafelyUnwrapped
    return (image.data.reshaped(to: [1, 784]), image.label)
    
}

In [6]:
let (input, label) = get_input()

Loading resource: train-images-idx3-ubyte
Loading resource: train-labels-idx1-ubyte
2020-02-05 03:48:24.174079: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 188160000 exceeds 10% of system memory.
Loading resource: t10k-images-idx3-ubyte
Loading resource: t10k-labels-idx1-ubyte


## Encrypted Inference

In [7]:
let input_private = PrivateTensor.from_values(values: Tensor<Double>(input))
let w1_private = PrivateTensor.from_values(values: Tensor<Double>(w1))
let b1_private = PrivateTensor.from_values(values: Tensor<Double>(b1))

In [8]:
let private_prediction = add(x: matmul(x: input_private, y: w1_private), y: b1_private)

In [9]:
let decode_prediction = private_prediction.reveal().decode()

In [10]:
print("Private prediction: ", decode_prediction)

Private prediction:  [[  14.44901691815272, -10.358481938728852, 0.36655997561347353,  -4.184727937814357,
   -8.945130315500686,   4.450845907636031,   5.928516994360616,  -1.699588477366255,
   2.4968754762993446, -2.6372504191434234]]


In [11]:
let public_prediction = Raw.add(Raw.matMul(input, w1), b1)
print("Public prediction: ", public_prediction)

Public prediction:  [[ 14.457324, -10.367435, 0.36947906,  -4.184661,  -8.950744,   4.455301,  5.9329185, -1.7006569,
   2.4987278, -2.6387553]]
